In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Mar 20 11:03:12 2019

@author: kaandorp, erikvansebille
"""

import numpy as np
import glob
import pandas as pd
import os
import progressbar

fileList = list(np.sort(glob.glob('/Volumes/oceanparcels/input_data/GlobalFishingWatch/FishingEffort/daily_csvs/*')))

galapagos_extent = [-91.8, -89, -1.4, 0.7]
# dx, dy = 7, 5
# lon_min = galapagos_extent[0] - dx
# lon_max = galapagos_extent[1] + dx
# lat_min = galapagos_extent[2] - dy
# lat_max = galapagos_extent[3] + dy

lon_min, lon_max, lat_min, lat_max = [-102, -75, -15, 8]
lon_edge = np.linspace(lon_min,lon_max,int((lon_max - lon_min + 0.1)*10) )
lat_edge = np.linspace(lat_min,lat_max,int((lat_max - lat_min + 0.1)*10) )

fishinghours_mat = np.zeros([len(lon_edge),len(lat_edge)])

pbar = progressbar.ProgressBar()
for i1 in pbar(range(len(fileList))):
    data = pd.read_csv(fileList[i1])
    lats = data.loc[:,'lat_bin']/10
    lons = data.loc[:,'lon_bin']/10
    fh = data.loc[:,'fishing_hours']
    
    indices_galapagos = ((lons >= lon_min) & (lons <= lon_max)) & ((lats >= lat_min) & (lats <= lat_max))
    
    lats_galapagos = lats[indices_galapagos]
    lons_galapagos = lons[indices_galapagos]
    fh_galapagos = fh[indices_galapagos]
    
    for i2 in range(len(lats_galapagos)):
        index_lon = np.where(abs(lon_edge-lons_galapagos.iloc[i2]) < 0.00001)  
        index_lat = np.where(abs(lat_edge-lats_galapagos.iloc[i2]) < 0.00001) 
        fishinghours_mat[index_lon,index_lat] += fh_galapagos.iloc[i2]

np.savez('fishingeffort.npz', fishinghours_mat, fishinghours_x, fishinghours_y)